In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
subject = 'Sergipe - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [5]:
data = pd.read_csv('../Inputs Atuais/2008_01_model_input_SE.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data = data.drop('Sergipe - Produção de Cimento (t)', axis=1)
data

,Ano,Sergipe - PIB - Estadual,Sergipe - PIB - Construção Civil,Sergipe - PIB - Per Capita,Sergipe - PIB - Preços de Mercado,Sergipe - IDH,Sergipe - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Sergipe - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Sergipe - IDH Educacao,Sergipe - Consumo de Cimento (t)
0,2008,2.275243e+07,1.304488e+06,9.995682,2.046863e+07,0.684816,0.671003,5.483335e+08,2.546644e+09,1.874883e+10,...,8.151229,12.692687,1013.229689,2586.438280,26.686275,23.696582,79.520472,3.343797,0.620749,391.414000
1,2009,2.284202e+07,1.307162e+06,9.962429,2.041487e+07,0.686798,0.671628,5.302764e+08,2.679945e+09,2.055015e+10,...,8.181780,14.565587,1013.167328,1929.165982,26.816394,23.060223,77.670883,3.361346,0.624297,404.787000
2,2010,2.337741e+07,1.316094e+06,9.971938,2.052469e+07,0.688421,0.671874,5.046425e+08,2.891930e+09,2.342247e+10,...,8.145498,16.790973,1013.363227,1655.291909,26.715685,21.890153,74.115505,3.285878,0.627591,480.282000
3,2011,2.381828e+07,1.308563e+06,9.960908,2.056187e+07,0.689694,0.671748,4.854622e+08,3.091988e+09,2.604265e+10,...,8.132819,17.733809,1013.713445,1515.293481,26.636305,20.489803,70.722822,3.172559,0.630627,579.482000
4,2012,2.420222e+07,1.297619e+06,9.936855,2.057944e+07,0.690538,0.671098,4.867661e+08,3.189506e+09,2.708753e+10,...,11.832646,19.761559,1013.891804,1451.567020,26.670892,19.276376,68.444205,3.096016,0.633403,646.084000
5,2013,2.449152e+07,1.275643e+06,9.895912,2.055335e+07,0.691083,0.670440,5.225848e+08,3.093872e+09,2.523394e+10,...,12.530554,21.193985,1014.116250,1407.100074,26.696079,19.374702,68.963603,3.006992,0.635603,590.339000
6,2014,2.470591e+07,1.254702e+06,9.861955,2.051042e+07,0.691757,0.669226,5.676413e+08,2.779340e+09,1.954361e+10,...,13.099831,20.010392,1014.355116,1389.188275,26.773639,19.486560,67.782988,2.799381,0.638737,633.190143
7,2015,2.484109e+07,1.238716e+06,9.830307,2.046045e+07,0.691403,0.667662,5.179392e+08,2.561731e+09,1.327596e+10,...,13.605605,18.966088,1014.406517,1396.519093,26.838598,19.578296,66.497486,2.216394,0.639881,592.014514
8,2016,2.492621e+07,1.222457e+06,9.814014,2.043197e+07,0.700162,0.672766,4.524144e+08,2.516859e+09,1.011370e+10,...,13.929548,18.086986,1014.355535,1396.331304,26.770657,19.675461,66.457655,1.943788,0.650966,552.624343
9,2017,2.497554e+07,1.205785e+06,9.822126,2.044513e+07,0.700449,0.671836,4.213689e+08,2.497710e+09,8.456309e+09,...,13.725374,18.116372,1014.300674,1352.042945,26.799279,19.803363,66.478178,2.248768,0.653326,473.789000


In [6]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Sergipe - PIB - Estadual,Sergipe - PIB - Construção Civil,Sergipe - PIB - Per Capita,Sergipe - PIB - Preços de Mercado,Sergipe - IDH,Sergipe - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Sergipe - IDH Longevidade,Sergipe - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Sergipe - IDH Educacao
0,2.275243e+07,1.304488e+06,9.995682,2.046863e+07,0.684816,0.671003,5.483335e+08,2.546644e+09,1.874883e+10,2.351319e+10,...,0.769402,8.151229,12.692687,1013.229689,2586.438280,26.686275,23.696582,79.520472,3.343797,0.620749
1,2.284202e+07,1.307162e+06,9.962429,2.041487e+07,0.686798,0.671628,5.302764e+08,2.679945e+09,2.055015e+10,2.446518e+10,...,0.771521,8.181780,14.565587,1013.167328,1929.165982,26.816394,23.060223,77.670883,3.361346,0.624297
2,2.337741e+07,1.316094e+06,9.971938,2.052469e+07,0.688421,0.671874,5.046425e+08,2.891930e+09,2.342247e+10,2.606035e+10,...,0.773103,8.145498,16.790973,1013.363227,1655.291909,26.715685,21.890153,74.115505,3.285878,0.627591
3,2.381828e+07,1.308563e+06,9.960908,2.056187e+07,0.689694,0.671748,4.854622e+08,3.091988e+09,2.604265e+10,2.845738e+10,...,0.774160,8.132819,17.733809,1013.713445,1515.293481,26.636305,20.489803,70.722822,3.172559,0.630627
4,2.420222e+07,1.297619e+06,9.936855,2.057944e+07,0.690538,0.671098,4.867661e+08,3.189506e+09,2.708753e+10,3.181492e+10,...,0.774664,11.832646,19.761559,1013.891804,1451.567020,26.670892,19.276376,68.444205,3.096016,0.633403
5,2.449152e+07,1.275643e+06,9.895912,2.055335e+07,0.691083,0.670440,5.225848e+08,3.093872e+09,2.523394e+10,3.629162e+10,...,0.774636,12.530554,21.193985,1014.116250,1407.100074,26.696079,19.374702,68.963603,3.006992,0.635603
6,2.470591e+07,1.254702e+06,9.861955,2.051042e+07,0.691757,0.669226,5.676413e+08,2.779340e+09,1.954361e+10,4.143397e+10,...,0.774272,13.099831,20.010392,1014.355116,1389.188275,26.773639,19.486560,67.782988,2.799381,0.638737
7,2.484109e+07,1.238716e+06,9.830307,2.046045e+07,0.691403,0.667662,5.179392e+08,2.561731e+09,1.327596e+10,4.469706e+10,...,0.773462,13.605605,18.966088,1014.406517,1396.519093,26.838598,19.578296,66.497486,2.216394,0.639881
8,2.492621e+07,1.222457e+06,9.814014,2.043197e+07,0.700162,0.672766,4.524144e+08,2.516859e+09,1.011370e+10,4.640544e+10,...,0.784003,13.929548,18.086986,1014.355535,1396.331304,26.770657,19.675461,66.457655,1.943788,0.650966
9,2.497554e+07,1.205785e+06,9.822126,2.044513e+07,0.700449,0.671836,4.213689e+08,2.497710e+09,8.456309e+09,4.607888e+10,...,0.783185,13.725374,18.116372,1014.300674,1352.042945,26.799279,19.803363,66.478178,2.248768,0.653326


In [7]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     404.787000
1     480.282000
2     579.482000
3     646.084000
4     590.339000
5     633.190143
6     592.014514
7     552.624343
8     473.789000
9     372.210000
10    409.903000
11    526.396000
12    511.494000
13    425.370000
14           NaN
Name: Sergipe - Consumo de Cimento (t), dtype: float64

In [8]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Sergipe - PIB - Estadual,Sergipe - PIB - Construção Civil,Sergipe - PIB - Per Capita,Sergipe - PIB - Preços de Mercado,Sergipe - IDH,Sergipe - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Sergipe - IDH Longevidade,Sergipe - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Sergipe - IDH Educacao
0,-1.853967,0.985736,1.564403,-0.328372,-1.484238,-0.055169,1.204752,-0.990234,0.166091,-1.487840,...,-1.473167,-1.379428,-2.332251,-1.601508,2.921375,-0.953904,2.093748,1.992324,1.179694,-1.544673
1,-1.741077,1.046118,1.056829,-1.318724,-1.122666,0.411794,0.820657,-0.546688,0.428911,-1.378556,...,-1.014740,-1.366996,-1.506429,-1.744422,1.053586,0.562151,1.697171,1.627416,1.216235,-1.238766
2,-1.066520,1.247888,1.201979,0.704334,-0.826555,0.595931,0.275392,0.158672,0.847995,-1.195437,...,-0.672413,-1.381760,-0.525185,-1.295478,0.275310,-0.611236,0.967987,0.925970,1.059099,-0.954737
3,-0.511040,1.077778,1.033617,1.389206,-0.594315,0.501385,-0.132596,0.824346,1.230291,-0.920269,...,-0.443833,-1.386920,-0.109458,-0.492880,-0.122527,-1.536117,0.095293,0.256622,0.823156,-0.692862
4,-0.027298,0.830558,0.666485,1.712873,-0.440345,0.015898,-0.104860,1.148829,1.382743,-0.534838,...,-0.334747,0.118665,0.784642,-0.084132,-0.303621,-1.133132,-0.660911,-0.192930,0.663783,-0.453547
5,0.337205,0.334142,0.041545,1.232130,-0.340955,-0.476154,0.657046,0.830615,1.112297,-0.020933,...,-0.340807,0.402667,1.416245,0.430234,-0.429984,-0.839667,-0.599635,-0.090457,0.478422,-0.263819
6,0.607332,-0.138902,-0.476774,0.441307,-0.218082,-1.382832,1.615453,-0.215960,0.282052,0.569384,...,-0.419492,0.634325,0.894361,0.977646,-0.480884,0.064000,-0.529926,-0.323382,0.046150,0.006394
7,0.777643,-0.500010,-0.959834,-0.479181,-0.282585,-2.551876,0.558230,-0.940033,-0.632425,0.943972,...,-0.594885,0.840142,0.433894,1.095443,-0.460052,0.820856,-0.472756,-0.577001,-1.167704,0.105104
8,0.884898,-0.867272,-1.208523,-1.003804,1.315140,1.262203,-0.835563,-1.089341,-1.093812,1.140087,...,1.685884,0.971965,0.046269,0.978606,-0.460585,0.029259,-0.412203,-0.584859,-1.735304,1.060937
9,0.947050,-1.243880,-1.084715,-0.761339,1.367575,0.566995,-1.495939,-1.153056,-1.335633,1.102599,...,1.508921,0.888880,0.059226,0.852881,-0.586441,0.362748,-0.332495,-0.580810,-1.100296,1.264488


In [9]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0     404.787000
1     480.282000
2     579.482000
3     646.084000
4     590.339000
5     633.190143
6     592.014514
7     552.624343
8     473.789000
9     372.210000
10    409.903000
11    526.396000
Name: Sergipe - Consumo de Cimento (t), dtype: float64

In [10]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Sergipe - PIB - Estadual,Sergipe - PIB - Construção Civil,Sergipe - PIB - Per Capita,Sergipe - PIB - Preços de Mercado,Sergipe - IDH,Sergipe - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Sergipe - IDH Longevidade,Sergipe - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Sergipe - IDH Educacao
12,0.398395,-0.999064,-0.787441,-1.136938,1.050925,-0.271725,1.108405,2.635823,-0.189959,0.726405,...,0.357602,0.721754,0.435649,1.373376,-0.585913,1.326673,-1.855589,-1.722974,-0.675291,1.2951


In [11]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    511.494
Name: Sergipe - Consumo de Cimento (t), dtype: float64

In [12]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-1:], train_target.iloc[-1:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [13]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [15]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[2250717880, 2573083406, 4045293253, 1208535543, 3106579724, 1608209788, 3764752279, 3967387144, 1101057797, 3363710057, 1666198883, 525468998, 3035569466, 3669192694, 2194065726, 1834126429, 3229692524, 3702275511, 2798382419, 3607407944, 3030365785, 2638078108, 1288590492, 3578780242, 3196250442, 460213545, 1285654977, 2951691923, 2809852848, 2573423589, 2375215960, 3160094511, 3586343098, 1270277475, 4019704728, 3818494664, 245196714, 1945366637, 2354083785, 3484760025, 4053634848, 3618079901, 91437678, 1451485172, 3157302964, 364262027, 2703097631, 1507705866, 1500141212, 2271704706, 2901636264, 1350440086, 1761827724, 4158275105, 1624143259, 1904673037, 1240315607, 28038495, 200414822, 3048163848, 822986751, 1095276657, 3434496784, 2241581747, 3081902608, 3126154097, 772190113, 2254332654, 1802767679, 4242434141, 3211766421, 1663596230, 1590424444, 866749919, 2848711772, 1668326013, 1478381361, 2589706536, 2186358263, 4029615218, 1190567879, 337770166, 1504133173, 3477698007, 5969

2023-09-15 21:35:31.393956: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-09-15 21:35:31.425295: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-09-15 21:35:31.456980: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-09-15 21:35:31.991548: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 2708.6865234375
winner_seed: 2250717880


Step: 1 ___________________________________________
val_loss: 6260.74169921875


Step: 2 ___________________________________________
val_loss: 1174.2083740234375
winner_seed: 4045293253


Step: 3 ___________________________________________
val_loss: 85.97853088378906
winner_seed: 1208535543


Step: 4 ___________________________________________
val_loss: 5210.8369140625


Step: 5 ___________________________________________
val_loss: 6502.4033203125


Step: 6 ___________________________________________
val_loss: 85.25000762939453
winner_seed: 3764752279


Step: 7 ___________________________________________
val_loss: 46.07810974121094
winner_seed: 3967387144


Step: 8 ___________________________________________
val_loss: 1342.841796875


Step: 9 ___________________________________________
val_loss: 1442.8660888671875


Step: 10 ___________________________________________
val_loss: 3214.7529296875


Step: 11 ______________________________

val_loss: 5825.91259765625


Step: 99 ___________________________________________
val_loss: 5597.20263671875


final_seed: 200414822


In [16]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 479ms/step - loss: 280338.3438 - val_loss: 267884.1562
Epoch 2/10000
1/1 [==============================] - 0s 15ms/step - loss: 275318.0312 - val_loss: 195410.7031
Epoch 3/10000
1/1 [==============================] - 0s 14ms/step - loss: 247844.0781 - val_loss: 314108.8750
Epoch 4/10000
1/1 [==============================] - 0s 15ms/step - loss: 220237.7031 - val_loss: 304295.7188
Epoch 5/10000
1/1 [==============================] - 0s 14ms/step - loss: 234447.0781 - val_loss: 262479.4062
Epoch 6/10000
1/1 [==============================] - 0s 14ms/step - loss: 236810.0469 - val_loss: 382307.3438
Epoch 7/10000
1/1 [==============================] - 0s 15ms/step - loss: 229201.2969 - val_loss: 314825.1562
Epoch 8/10000
1/1 [==============================] - 0s 15ms/step - loss: 223777.5156 - val_loss: 360603.3438
Epoch 9/10000
1/1 [==============================] - 0s 16ms/step - loss: 217257.8125 - val_loss: 332337.3438
Epoch 10/

1/1 [==============================] - 0s 14ms/step - loss: 65996.2109 - val_loss: 114755.8594
Epoch 76/10000
1/1 [==============================] - 0s 15ms/step - loss: 64806.8320 - val_loss: 112622.9375
Epoch 77/10000
1/1 [==============================] - 0s 14ms/step - loss: 63800.4805 - val_loss: 111334.6172
Epoch 78/10000
1/1 [==============================] - 0s 14ms/step - loss: 62722.5312 - val_loss: 109386.5000
Epoch 79/10000
1/1 [==============================] - 0s 15ms/step - loss: 61698.1055 - val_loss: 107835.2344
Epoch 80/10000
1/1 [==============================] - 0s 15ms/step - loss: 60690.4258 - val_loss: 106226.8203
Epoch 81/10000
1/1 [==============================] - 0s 15ms/step - loss: 59743.3008 - val_loss: 104936.2578
Epoch 82/10000
1/1 [==============================] - 0s 15ms/step - loss: 58737.2969 - val_loss: 103116.1719
Epoch 83/10000
1/1 [==============================] - 0s 15ms/step - loss: 57757.6719 - val_loss: 101414.6641
Epoch 84/10000
1/1 [=====

Epoch 150/10000
1/1 [==============================] - 0s 16ms/step - loss: 21108.5410 - val_loss: 40222.5508
Epoch 151/10000
1/1 [==============================] - 0s 15ms/step - loss: 20855.1895 - val_loss: 39819.2734
Epoch 152/10000
1/1 [==============================] - 0s 14ms/step - loss: 20544.5020 - val_loss: 39188.3398
Epoch 153/10000
1/1 [==============================] - 0s 14ms/step - loss: 19840.4355 - val_loss: 39357.3008
Epoch 154/10000
1/1 [==============================] - 0s 15ms/step - loss: 19610.8184 - val_loss: 33610.3086
Epoch 155/10000
1/1 [==============================] - 0s 15ms/step - loss: 19925.1621 - val_loss: 38680.4023
Epoch 156/10000
1/1 [==============================] - 0s 16ms/step - loss: 19504.3828 - val_loss: 37086.2891
Epoch 157/10000
1/1 [==============================] - 0s 16ms/step - loss: 19245.1328 - val_loss: 36814.1172
Epoch 158/10000
1/1 [==============================] - 0s 15ms/step - loss: 18725.2695 - val_loss: 37503.2969
Epoch 159/

1/1 [==============================] - 0s 15ms/step - loss: 9710.4258 - val_loss: 17444.6641
Epoch 225/10000
1/1 [==============================] - 0s 15ms/step - loss: 9601.7529 - val_loss: 17106.4414
Epoch 226/10000
1/1 [==============================] - 0s 16ms/step - loss: 9563.1143 - val_loss: 17107.1680
Epoch 227/10000
1/1 [==============================] - 0s 15ms/step - loss: 9459.1729 - val_loss: 16779.8398
Epoch 228/10000
1/1 [==============================] - 0s 15ms/step - loss: 9393.7686 - val_loss: 16671.1543
Epoch 229/10000
1/1 [==============================] - 0s 14ms/step - loss: 9323.1143 - val_loss: 16484.2930
Epoch 230/10000
1/1 [==============================] - 0s 15ms/step - loss: 9256.4443 - val_loss: 16323.8057
Epoch 231/10000
1/1 [==============================] - 0s 16ms/step - loss: 9191.6904 - val_loss: 16160.0752
Epoch 232/10000
1/1 [==============================] - 0s 14ms/step - loss: 9128.1406 - val_loss: 16024.4785
Epoch 233/10000
1/1 [==============

1/1 [==============================] - 0s 17ms/step - loss: 6669.8203 - val_loss: 9224.7559
Epoch 300/10000
1/1 [==============================] - 0s 15ms/step - loss: 6670.7798 - val_loss: 9217.7520
Epoch 301/10000
1/1 [==============================] - 0s 14ms/step - loss: 6633.2012 - val_loss: 9084.8799
Epoch 302/10000
1/1 [==============================] - 0s 15ms/step - loss: 6769.8359 - val_loss: 9285.7881
Epoch 303/10000
1/1 [==============================] - 0s 15ms/step - loss: 6603.2344 - val_loss: 8919.2598
Epoch 304/10000
1/1 [==============================] - 0s 14ms/step - loss: 6598.6665 - val_loss: 8989.7861
Epoch 305/10000
1/1 [==============================] - 0s 15ms/step - loss: 6565.7441 - val_loss: 8833.2861
Epoch 306/10000
1/1 [==============================] - 0s 15ms/step - loss: 6548.3237 - val_loss: 8792.2500
Epoch 307/10000
1/1 [==============================] - 0s 15ms/step - loss: 6535.8926 - val_loss: 8724.7676
Epoch 308/10000
1/1 [=======================

...

1/1 [==============================] - 0s 15ms/step - loss: 5054.7871 - val_loss: 3392.0137
Epoch 1509/10000
1/1 [==============================] - 0s 14ms/step - loss: 5053.3950 - val_loss: 3464.1387
Epoch 1510/10000
1/1 [==============================] - 0s 15ms/step - loss: 5051.7153 - val_loss: 3442.1670
Epoch 1511/10000
1/1 [==============================] - 0s 14ms/step - loss: 5051.0537 - val_loss: 3437.6350
Epoch 1512/10000
1/1 [==============================] - 0s 15ms/step - loss: 5050.9292 - val_loss: 3438.9663
Epoch 1513/10000
1/1 [==============================] - 0s 14ms/step - loss: 5051.0894 - val_loss: 3433.5566
Epoch 1514/10000
1/1 [==============================] - 0s 15ms/step - loss: 5051.7036 - val_loss: 3426.9756
Epoch 1515/10000
1/1 [==============================] - 0s 15ms/step - loss: 5052.4395 - val_loss: 3459.8616
Epoch 1516/10000
1/1 [==============================] - 0s 15ms/step - loss: 5050.4800 - val_loss: 3436.8047
Epoch 1517/10000
1/1 [==============

1/1 [==============================] - 0s 15ms/step - loss: 5487.2085 - val_loss: 4215.5078
Epoch 1584/10000
1/1 [==============================] - 0s 15ms/step - loss: 5451.7778 - val_loss: 4281.3354
Epoch 1585/10000
1/1 [==============================] - 0s 15ms/step - loss: 5609.7598 - val_loss: 5494.7012
Epoch 1586/10000
1/1 [==============================] - 0s 16ms/step - loss: 5530.7397 - val_loss: 5277.0327
Epoch 1587/10000
1/1 [==============================] - 0s 15ms/step - loss: 5631.1421 - val_loss: 5515.0293
Epoch 1588/10000
1/1 [==============================] - 0s 15ms/step - loss: 5457.9048 - val_loss: 5094.5171
Epoch 1589/10000
1/1 [==============================] - 0s 14ms/step - loss: 5527.0776 - val_loss: 5381.0786
Epoch 1590/10000
1/1 [==============================] - 0s 14ms/step - loss: 5486.3418 - val_loss: 5304.5894
Epoch 1591/10000
1/1 [==============================] - 0s 14ms/step - loss: 5518.4590 - val_loss: 4039.0508
Epoch 1592/10000
1/1 [==============

1/1 [==============================] - 0s 15ms/step - loss: 5282.9443 - val_loss: 4295.6646
Epoch 1659/10000
1/1 [==============================] - 0s 15ms/step - loss: 5280.3584 - val_loss: 4284.5908
Epoch 1660/10000
1/1 [==============================] - 0s 14ms/step - loss: 5278.0859 - val_loss: 4289.9741
Epoch 1661/10000
1/1 [==============================] - 0s 15ms/step - loss: 5275.9370 - val_loss: 4298.7295
Epoch 1662/10000
1/1 [==============================] - 0s 14ms/step - loss: 5274.2690 - val_loss: 4307.2334
Epoch 1663/10000
1/1 [==============================] - 0s 15ms/step - loss: 5276.7334 - val_loss: 4335.4072
Epoch 1664/10000
1/1 [==============================] - 0s 15ms/step - loss: 5269.9058 - val_loss: 4303.4771
Epoch 1665/10000
1/1 [==============================] - 0s 15ms/step - loss: 5267.5366 - val_loss: 4298.1372
Epoch 1666/10000
1/1 [==============================] - 0s 15ms/step - loss: 5265.6616 - val_loss: 4316.5195
Epoch 1667/10000
1/1 [==============

1/1 [==============================] - 0s 14ms/step - loss: 5149.1860 - val_loss: 4493.0576
Epoch 1734/10000
1/1 [==============================] - 0s 15ms/step - loss: 5148.0298 - val_loss: 4513.3525
Epoch 1735/10000
1/1 [==============================] - 0s 15ms/step - loss: 5146.5605 - val_loss: 4513.0078
Epoch 1736/10000
1/1 [==============================] - 0s 19ms/step - loss: 5144.8262 - val_loss: 4504.7412
Epoch 1737/10000
1/1 [==============================] - 0s 15ms/step - loss: 5143.3911 - val_loss: 4508.8882
Epoch 1738/10000
1/1 [==============================] - 0s 15ms/step - loss: 5142.3394 - val_loss: 4525.7769
Epoch 1739/10000
1/1 [==============================] - 0s 16ms/step - loss: 5140.5630 - val_loss: 4512.7456
Epoch 1740/10000
1/1 [==============================] - 0s 15ms/step - loss: 5139.1333 - val_loss: 4525.2432
Epoch 1741/10000
1/1 [==============================] - 0s 15ms/step - loss: 5137.7104 - val_loss: 4521.0850
Epoch 1742/10000
1/1 [==============

1/1 [==============================] - 0s 15ms/step - loss: 5059.3979 - val_loss: 4749.0469
Epoch 1809/10000
1/1 [==============================] - 0s 15ms/step - loss: 5058.4229 - val_loss: 4752.6567
Epoch 1810/10000
1/1 [==============================] - 0s 15ms/step - loss: 5057.4531 - val_loss: 4755.9644
Epoch 1811/10000
1/1 [==============================] - 0s 15ms/step - loss: 5056.4380 - val_loss: 4754.5967
Epoch 1812/10000
1/1 [==============================] - 0s 15ms/step - loss: 5055.5317 - val_loss: 4763.8560
Epoch 1813/10000
1/1 [==============================] - 0s 14ms/step - loss: 5055.0522 - val_loss: 4774.1655
Epoch 1814/10000
1/1 [==============================] - 0s 15ms/step - loss: 5053.6284 - val_loss: 4767.6772
Epoch 1815/10000
1/1 [==============================] - 0s 15ms/step - loss: 5052.6421 - val_loss: 4768.1411
Epoch 1816/10000
1/1 [==============================] - 0s 20ms/step - loss: 5051.7402 - val_loss: 4777.0381
Epoch 1817/10000
1/1 [==============

In [17]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 50ms/step
[511.494](test_target) - [[527.71173]](prediction) = 16.217730957031222


In [18]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [19]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Sergipe - PIB - Estadual,Sergipe - PIB - Construção Civil,Sergipe - PIB - Per Capita,Sergipe - PIB - Preços de Mercado,Sergipe - IDH,Sergipe - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Sergipe - IDH Longevidade,Sergipe - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Sergipe - IDH Educacao
0,-0.863219,-0.959139,1.358624,-0.017092,-1.263295,-1.359965,1.148205,-1.122043,-1.121418,-1.109924,...,-1.279461,-0.519495,-1.188168,-0.290340,1.355077,-0.954425,1.088406,1.078152,0.417308,-1.239600
1,-0.538514,-0.420452,-1.019320,-1.216109,0.081135,0.344018,0.140893,-0.184474,-0.185492,-0.204018,...,0.117968,1.398867,-0.070139,-1.053486,-0.327076,1.380985,0.237800,0.253510,0.961556,0.030272
2,1.401733,1.379591,-0.339304,1.233202,1.182160,1.015947,-1.289098,1.306518,1.306910,1.313942,...,1.161492,-0.879372,1.258307,1.343826,-1.028001,-0.426561,-1.326206,-1.331661,-1.378864,1.209328


train_target:


,Sergipe - Consumo de Cimento (t)
0,404.787
1,480.282
2,579.482


test_input:


...

,Sergipe - PIB - Estadual,Sergipe - PIB - Construção Civil,Sergipe - PIB - Per Capita,Sergipe - PIB - Preços de Mercado,Sergipe - IDH,Sergipe - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Sergipe - IDH Longevidade,Sergipe - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Sergipe - IDH Educacao
12,0.398395,-0.999064,-0.787441,-1.136938,1.050925,-0.271725,1.108405,2.635823,-0.189959,0.726405,...,0.357602,0.721754,0.435649,1.373376,-0.585913,1.326673,-1.855589,-1.722974,-0.675291,1.2951


test_target:


,Sergipe - Consumo de Cimento (t)
12,511.494


1/1 [==============================] - 0s 35ms/step
Error: 16.217730957031222




[579.4801025390625,
 612.780029296875,
 511.19598388671875,
 630.91015625,
 569.813232421875,
 552.593505859375,
 474.223876953125,
 492.5788879394531,
 419.60394287109375,
 527.7117309570312]

In [20]:
display(targets)
display(predictions)

[646.0840000000001,
 590.339,
 633.1901428571429,
 592.0145142857143,
 552.6243428571429,
 473.789,
 372.21,
 409.903,
 526.396,
 511.494]

[579.4801025390625,
 612.780029296875,
 511.19598388671875,
 630.91015625,
 569.813232421875,
 552.593505859375,
 474.223876953125,
 492.5788879394531,
 419.60394287109375,
 527.7117309570312]

In [21]:
mae = mean_absolute_error(predictions, targets)
mae

65.36276760951449

In [22]:
porcentage = mae/np.mean(targets)
porcentage

0.12313908401948909